In [4]:
import mlflow
import pandas as pd
from transformers import pipeline
from tqdm.notebook import tqdm
import numpy as np
tqdm.pandas()
import datetime
from src.helpers import zero_shot_predict_single_model, get_top_n_label_and_score, save_dataframe_with_timestamp

In [5]:
EXPERIMENT_NAME = "zero_shot_prediction"
# EXPERIMENT_ID = mlflow.create_experiment(EXPERIMENT_NAME)
mlflow.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/JOSHUALE/Documents/Github/mlflow_tutorial/experiments/mlruns/227815192018524595', creation_time=1697818753176, experiment_id='227815192018524595', last_update_time=1697818753176, lifecycle_stage='active', name='zero_shot_prediction_L14D_001', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/JOSHUALE/Documents/Github/mlflow_tutorial/experiments/mlruns/677968303451148623', creation_time=1697816832103, experiment_id='677968303451148623', last_update_time=1697816832103, lifecycle_stage='active', name='zero_shot_prediction_001', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/JOSHUALE/Documents/Github/mlflow_tutorial/experiments/mlruns/0', creation_time=1697816832085, experiment_id='0', last_update_time=1697816832085, lifecycle_stage='active', name='Default', tags={}>]

### For sample experiment on small df

In [10]:
# for sample experiment:
EXPERIMENT_ID = '677968303451148623'
RUN_DESCRIPTION = ''
with mlflow.start_run(experiment_id=EXPERIMENT_ID, description=RUN_DESCRIPTION) as run:
    RUN_ID = run.info.run_id
    print(f"Started run: {RUN_ID}")
    # change path accordingly
    class_labels_data_path = '../data/input/class_label_by_topic.csv'
    class_labels_data = pd.read_csv(class_labels_data_path)
    candidate_labels = list(class_labels_data['CLASS_DESCRIPTION'])

    inference_input_data_path = '../temp_training/medallion/gold_NEWS_API_ML_LABELLED_summary.csv'
    inference_df = pd.read_csv(inference_input_data_path)
    inference_df = inference_df.sample(5, random_state=99) # getting a sample only

    model_path = '../models/pretrained/bart-large-mnli/'
    loaded_classifier = pipeline("zero-shot-classification", model=model_path)
    
    mlflow.log_param("model_path", model_path)
    mlflow.log_artifact(class_labels_data_path, "class_labels_data")
    mlflow.log_artifact(inference_input_data_path, "inference_input_data")
    
    inference_df['ZERO_SHOT_PREDICTION'] = inference_df.progress_apply(lambda row: 
                                                   zero_shot_predict_single_model(
                                                       classifier=loaded_classifier, 
                                                       sequence_to_classify=row['BODY_SUMMARY'], 
                                                       candidate_labels=candidate_labels), 
                                                    axis=1)
    inference_df['FIRST_PREDICTION_CLASS'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 1)[0])
    inference_df['FIRST_PREDICTION_SCORE'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 1)[1])
    inference_df['SECOND_PREDICTION_CLASS'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 2)[0])
    inference_df['SECOND_PREDICTION_SCORE'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 2)[1])
    # Save the DataFrame to a CSV file
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    predictions_path = f"./output/temp_mlflow/zero_shot_prediction_{current_datetime}.csv"
    inference_df.to_csv(predictions_path, index=False)
    # Log the CSV file as an artifact
    mlflow.log_artifact(predictions_path, f"zero_shot_prediction")
mlflow.end_run() 

Started run: cdb489738fe74707b16ee5f81f00aa7a


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/5 [00:00<?, ?it/s]

### For prediction on full dataset

In [12]:
# for time consuming experiment:
EXPERIMENT_ID = '227815192018524595'
RUN_DESCRIPTION = ''
with mlflow.start_run(experiment_id=EXPERIMENT_ID, description=RUN_DESCRIPTION) as run:
    RUN_ID = run.info.run_id
    print(f"Started run: {RUN_ID}")
    # change path accordingly
    class_labels_data_path = '../data/input/class_label_by_topic.csv'
    class_labels_data = pd.read_csv(class_labels_data_path)
    candidate_labels = list(class_labels_data['CLASS_DESCRIPTION'])

    inference_input_data_path = '../temp_training/medallion/gold_NEWS_API_ML_LABELLED_summary.csv'
    inference_df = pd.read_csv(inference_input_data_path)
    # inference_df = inference_df.sample(5, random_state=99) # getting a sample only

    model_path = '../models/pretrained/bart-large-mnli/'
    loaded_classifier = pipeline("zero-shot-classification", model=model_path)
    
    mlflow.log_param("model_path", model_path)
    mlflow.log_artifact(class_labels_data_path, "class_labels_data")
    mlflow.log_artifact(inference_input_data_path, "inference_input_data")
    
    inference_df['ZERO_SHOT_PREDICTION'] = inference_df.progress_apply(lambda row: 
                                                   zero_shot_predict_single_model(
                                                       classifier=loaded_classifier, 
                                                       sequence_to_classify=row['BODY_SUMMARY'], 
                                                       candidate_labels=candidate_labels), 
                                                    axis=1)
    inference_df['FIRST_PREDICTION_CLASS'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 1)[0])
    inference_df['FIRST_PREDICTION_SCORE'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 1)[1])
    inference_df['SECOND_PREDICTION_CLASS'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 2)[0])
    inference_df['SECOND_PREDICTION_SCORE'] = inference_df['ZERO_SHOT_PREDICTION'].apply(lambda result_dict: get_top_n_label_and_score(result_dict, 2)[1])
    # Save the DataFrame to a CSV file
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    predictions_path = f"./output/temp_mlflow/zero_shot_prediction_{current_datetime}.csv"
    inference_df.to_csv(predictions_path, index=False)
    # Log the CSV file as an artifact
    mlflow.log_artifact(predictions_path, f"zero_shot_prediction")
mlflow.end_run() 

    

Started run: 647dfdc244ad4e1eacbdb89518721ffd


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/71 [00:00<?, ?it/s]